In [ ]:
# app/load_data.py
import csv
from .database import SessionLocal, engine, Base
from . import models, crud, schemas

def init_db():
    Base.metadata.create_all(bind=engine)

def load_csv(path="data/bank_branches.csv"):
    db = SessionLocal()
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # adapt these keys depending on CSV columns
            bank_name = row.get("bank_name") or row.get("bank")
            bank_id = row.get("bank_id")
            ifsc = row.get("ifsc")
            branch = row.get("branch")
            address = row.get("address")
            city = row.get("city")
            district = row.get("district")
            state = row.get("state")

            # get-or-create bank
            bank = db.query(models.Bank).filter(models.Bank.name == bank_name).first()
            if not bank:
                bank = models.Bank(name=bank_name, bank_id=int(bank_id) if bank_id else None)
                db.add(bank)
                db.commit()
                db.refresh(bank)
            # skip if duplicate IFSC
            exists = db.query(models.Branch).filter(models.Branch.ifsc == ifsc).first()
            if exists:
                continue
            branch_obj = models.Branch(
                ifsc=ifsc, branch=branch, address=address,
                city=city, district=district, state=state,
                bank_id_fk=bank.id
            )
            db.add(branch_obj)
        db.commit()
    db.close()

if __name__ == "__main__":
    init_db()
    load_csv()
    print("Data loaded.")